# Data Analysis - Velib project

We consider the [velib](https://www.velib-metropole.fr/donnees-open-data-gbfs-du-service-velib-metropole) data set, related to the bike sharing system of Paris. The data are loading profiles of the bike stations over one week, collected every hour, from the period Monday 2nd Sept. - Sunday 7th Sept., 2014. The loading profile of a station, or simply loading, is defined as the ratio of number of available bikes divided by the number of bike docks. A loading of 1 means that the station is fully loaded, i.e. all bikes are available. A loading of 0 means that the station is empty, all bikes have been rent.

From the viewpoint of data analysis, the individuals are the stations. The variables are the 168 time steps (hours in the week). **The aim is to detect clusters in the data, corresponding to common customer usages.** This clustering should then be used to predict the loading profile.

# <FONT COLOR="Red">Loading the data   </font>

In [ ]:
options(warn = -1)
library(ggplot2)
library(reshape2)
library(gridExtra)
library(ggmap)
library(dplyr)
library(RColorBrewer)
library(tidyverse)
library(GGally)
library(plotly)
library(corrplot)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)
library(cluster)
library(scales)
library(stats)
library(broom)

In [ ]:
load('velib_data/velib.RData')
summary(velib)

In [ ]:
# data preparation
loading = as.matrix(velib$data)
colnames(loading) = 1:ncol(loading)
rownames(loading) = velib$names

stations = 1:nrow(loading)
coord = velib$position[stations,]
coord$bonus = velib$bonus[stations]

# select exactly 7 days of data (we remove the first 13 dates)
dates = 14:181
loading = loading[stations, dates]
colnames(loading) = 1:length(dates)

head(loading)
head(coord)

In [ ]:
# Data loading
#load <- read.csv("velib_data/velibLoading.csv", sep = " ")
#coord <- read.csv("velib_data/velibCoord.csv", sep = " ")
#head(load)

In [ ]:
#head(coord)
#summary(coord)

# <FONT COLOR="Red">Data verification   </font>

In [ ]:
# Check for missing data
load_miss <- sum(colSums(is.na(loading)) > 0)
print(load_miss)
coord_miss <- sum(colSums(is.na(coord)) > 0)
print(coord_miss)

In [ ]:
# Check for duplicates
load_dupl <- sum(duplicated(loading))
print(load_dupl)
coord_dupl <- sum(duplicated(coord))
print(coord_dupl)

In [ ]:
# Stations in descending order of occurrence
station_names <- as.data.frame(table(velib$names))
station_names <- station_names[order(-station_names$Freq), ]
paste("number of stations present more than once:", sum(station_names$Freq > 1))
station_names

In [ ]:
# Display the station with the most occurrences
name <- station_names$Var1[1]
name
coord[velib$names == name, ]

In [ ]:
# Plotting loading profiles of station with the most occurence

options(repr.plot.width = 15, repr.plot.height = 10)

#--------------------#

timeTick = 1 + 24*(0:6)  # vector corresponding to the beginning of days

stations$time_range = 1:ncol(loading)

p = list()
for (i in 1:3){
    dfi = melt(loading[velib$names == station_names$Var1[1], ][i,])
    p[[i]] = ggplot(dfi, aes(x=stations$time_range, y=value)) + 
        geom_line(col="darkorchid") + 
        geom_vline(xintercept=timeTick, col="orange", linetype="dashed")
}
do.call(grid.arrange,p)

# <FONT COLOR="Red">Data visualization   </font>

In [ ]:
# Plottng loading profile

options(repr.plot.width = 15, repr.plot.height = 6)

#---------------------#

time_tick = 1 + 24*(0:6)  # vector corresponding to the beginning of days

# select a station
i = sample.int(nrow(loading), 1)

df = melt(loading[i,])  #the function melt reshapes it from wide to long
df$time_range = 1:ncol(loading)

ggplot(df, aes(x=time_range, y=value)) + geom_line(col="darkorchid") +
    geom_vline(xintercept=time_tick, col="orange", linetype="dashed") +
    labs(title=velib$names[i])

In [ ]:
# Plotting loading profile for 16 stations in order to compare

options(repr.plot.width = 15, repr.plot.height = 10)

#--------------------------#

timeTick = 1 + 24*(0:6)  # vector corresponding to the beginning of days

# select 16 stations
stations = sample.int(nrow(loading), 16)

df = melt(loading[stations,])  #the function melt reshapes it from wide to long

p = list()
for (i in 1:16){
    dfi = df[df$Var1 == velib$names[stations[i]],]
    p[[i]] = ggplot(dfi, aes(x=Var2, y=value)) + 
        geom_line(col="darkorchid") + 
        geom_vline(xintercept=timeTick, col="orange", linetype="dashed") +
        labs(title=velib$names[stations[i]])
}
do.call(grid.arrange,p)

In [ ]:
# Boxplot of variables

options(repr.plot.width = 15, repr.plot.height = 6)

#-------------------#

# Total data
df = melt(loading)
p1 = ggplot(df, aes(x=Var2, y=value, fill=Var1)) + 
    geom_boxplot(alpha=.3, show.legend = FALSE) +
    xlab('') + ylab('') + theme_minimal()


# Only a third of the data
df = melt(loading[ seq(from=1, to=+floor(nrow(loading)/3)) ,])
p2 = ggplot(df, aes(x=Var2, y=value, fill=Var1)) + 
    geom_boxplot(alpha=.3, show.legend = FALSE) +
    xlab('') + ylab('')

# The next third
df = melt(loading[ seq(from=1+floor(nrow(loading)/3), to=2*floor(nrow(loading)/3)) ,])
p3 = ggplot(df, aes(x=Var2, y=value, fill=Var1)) + 
    geom_boxplot(alpha=.3, show.legend = FALSE) +
    xlab('') + ylab('')

# The remaining third
df = melt(loading[ seq(from=1+2*floor(nrow(loading)/3), to=nrow(loading)) ,])
p4 = ggplot(df, aes(x=Var2, y=value, fill=Var1)) + 
    geom_boxplot(alpha=.3, show.legend = FALSE) +
    xlab('') + ylab('')


grid.arrange(p1, p2, p3, p4, nrow=4)

In [ ]:
# Average fill rate

print('--- Average fill rate ---')
print(mean(loading))


print('--- Least crowded station, on average ---')
i = which.min(rowMeans(loading)) 
print(rowMeans(loading)[i])


print('--- Fullest station, on average ---')
i = which.max(rowMeans(loading))
print(rowMeans(loading)[i])

In [ ]:
# Average fill rate between stations

options(repr.plot.width = 15, repr.plot.height = 6)

#-------------------#

df = data.frame(stations = c(1:nrow(loading)), mean = rowMeans(loading))
ggplot(df, aes(x = stations, y= mean)) + 
    geom_line(color = 'cornflowerblue', linewidth=1) +
    geom_hline(yintercept = mean(loading), color = 'darkorange', linewidth=2) +
    labs(x = "Stations", y = "Average loading")

In [ ]:
# Plotting the laoding mean per hour for each day of the week

options(repr.plot.width = 15, repr.plot.height = 6)

#-------------------#

mean_per_hour_per_day = colMeans(loading)
mean_per_hour_per_day = matrix(mean_per_hour_per_day, nrow = 24)
mean_per_hour         = rowMeans(mean_per_hour_per_day)

#------------------#

mean_per_hour_per_day            = as.data.frame(mean_per_hour_per_day)
colnames(mean_per_hour_per_day)  = list("Monday", "Tuesday", "Wednesday","Thursday", "Friday", "Saturday", "Sunday")
mean_per_hour_per_day$time_range = c(1:24)
mean_per_hour_per_day            = melt(mean_per_hour_per_day, id='time_range', variable.name='Days')

mean_per_hour            = as.data.frame(mean_per_hour)
colnames(mean_per_hour)  = list("Weekly")
mean_per_hour$time_range = c(1:24)

#-----------------#

ggplot() + 
    geom_line(data=mean_per_hour_per_day, aes(x=time_range, y=value, color=Days)) + 
    geom_line(data=mean_per_hour, aes(x=time_range, y=Weekly), linewidth = 1.5)

## Influence of the day/time of the day

In [ ]:
# Average loading for day and night

data_df = loading
# Create a dataframe for night hours
data_nuit <- data_df[, which((1:ncol(data_df) %% 24) >= 22 | (1:ncol(data_df) %% 24) < 6)]

# Create a dataframe for day hours
data_jour <- data_df[, which((1:ncol(data_df) %% 24) >= 6 & (1:ncol(data_df) %% 24) < 22)]

# Calculate average total overnight fill rate
moyenne_totale_nuit <- mean(unlist(data_nuit))

# Calculate the total average fill rate on the day
moyenne_totale_jour <- mean(unlist(data_jour))

cat(paste(" average loading during the night :", moyenne_totale_nuit, "\n average loading during the day :", moyenne_totale_jour, "\n"))

In [ ]:
# Plotting velib stations on Paris map for Monday at 6am, 12am and 11pm

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

hours = c(6, 12, 23)

dfi = coord
p = list()
for (i in 1:length(hours)){
    dfi$loading = loading[,hours[i]]
    p[[i]] = ggplot(dfi, aes(x=longitude, y=latitude, color=loading)) + 
        geom_point() +
        labs(title = paste("Stations loading - Monday",hours[i],"h"))
}

do.call(grid.arrange,c(p, ncol=2))

In [ ]:
# Plotting velib stations loading on Paris map at 6pm, for each day

options(repr.plot.width = 15, repr.plot.height = 15)

#-------------------#


h = 18
hours = seq(h, 168, 24)
days  = list("Monday", "Tuesday", "Wednesday","Thursday", "Friday", "Saturday", "Sunday")

dfi = coord
p = list()
for (i in 1:7){
    dfi$loading = loading[,hours[i]]
    p[[i]] = ggplot(dfi, aes(x=longitude, y=latitude, color=loading)) + 
        geom_point() +
        labs(title = paste("Stations loading - ", days[i], h,"h"))
}

do.call(grid.arrange,c(p, ncol=3))

## Influence of altitude

In [ ]:
# Repartition of Hill/Valley stations

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

df = data.frame(size=c(sum(coord$bonus==0), sum(coord$bonus==1)),
                labels = c('No hill','Hill'))

#-------------------#

ggplot(df, aes(x="", y=size, fill=labels)) +
    geom_bar(stat="identity", width=1) +
    geom_text(aes(label=size), position = position_stack(vjust = 0.5)) +
    coord_polar(theta = "y") +
    scale_color_hue(direction = -1) +  # to reverse the default colormap order
    theme_void()

In [ ]:
# Repartition of Hill/Valley stations on Paris map

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

coord$hill = as.factor(coord$bonus)
levels(coord$hill) = c('No hill','Hill')

#-------------------#

ggplot(coord, aes(x=longitude, y=latitude, color=hill)) + 
    geom_point() +
    scale_color_hue(direction = -1) +
    labs(title = 'Hilltop stations')

In [ ]:
# Visualization on the Paris map

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

register_stadiamaps("b567e1ef-c476-44d8-a1c6-38c49adf7440", write = TRUE)

#-------------------#

qmplot(data=coord, longitude, latitude, color=hill) +
    scale_color_hue(direction = -1) +
    labs(title = 'Hilltop stations')

In [ ]:
#Splitting the dataset

loading_hill  = loading[velib$bonus == 1]
loading_valley = loading[velib$bonus == 0]
loading_hill <- matrix(loading_hill, nrow = 127, ncol = 168, byrow = FALSE)
loading_valley <- matrix(loading_valley, nrow = 1062, ncol = 168, byrow = FALSE)

In [ ]:
# Average fill rate for Hill/Valley stations

mean_hill = mean(loading_hill)
mean_valley = mean(loading_valley)

print('--- Average fill rate for hilltop stations : ---')
mean_hill
print('--- Average fill rate for valley stations : ---')
mean_valley

In [ ]:
# Conversion to dataframes
loading_hill_df <- as.data.frame(loading_hill)
loading_valley_df <- as.data.frame(loading_valley)

# Calculating station averages
meanR_hill <- apply(loading_hill_df, 1, mean)
meanR_valley <- apply(loading_valley_df, 1, mean)

# Sort averages by station
meanR_hill_sorted <- sort(meanR_hill)
meanR_valley_sorted <- sort(meanR_valley)

#---------------------#

par(mfrow=c(1,2), figsize=c(10,5))

# Plot points for sorted and overall averages
plot(meanR_valley_sorted, type="o", pch=19, col="blue", xlab="Stations", ylab="Average loading", main="Average loading per station - Hill vs. Valley")
points(rep(mean_valley, length(meanR_valley)), col="blue")
points(meanR_hill_sorted, type="o", pch=19, col="red")
points(rep(mean_hill, length(meanR_hill)), col="red")

# Histogram creation
hist(meanR_valley, breaks=20, prob=TRUE, xlim=c(min(meanR_valley, meanR_hill), max(meanR_valley, meanR_hill)), col="lightblue", main="Histograms of average loading - Hill vs. Valley", xlab="Average Loading", ylab="Density")
hist(meanR_hill, breaks=20, prob=TRUE, add=TRUE, col="pink", alpha=0.5)
legend("topright", legend=c("Valley", "Hill"), fill=c("lightblue", "pink"), border=NA)


dev.off()


In [ ]:
# Print most loaded Hill stations

hill_coord <- coord[velib$bonus == 1, ]
hill_load <- loading[velib$bonus == 1, ]
h <- apply(hill_load, 1, mean)

sorted_indexes <- c()
                                               
for (s in 1:9) {
  i <- which.max(h)
  sorted_indexes <- c(sorted_indexes, i)
  if (s < 4) {
    print(paste("Nom :",rownames(hill_load)[i],"; Average fill rate :", h[i]))
    print(hill_coord[i, ])
  }
  h <- h[-i]
}

sorted_hill_coords <- hill_coord[sorted_indexes, ]
sorted_hill_load <- apply(hill_load, 1, mean)[sorted_indexes]


In [ ]:
# Plotting the loading mean per hour for each day of the week for Hill stations

hill_mean_per_hour_per_day = colMeans(loading_hill)
hill_mean_per_hour_per_day = matrix(hill_mean_per_hour_per_day, nrow = 24)
hill_mean_per_hour         = rowMeans(hill_mean_per_hour_per_day)

#---------------------#

hill_mean_per_hour_per_day            = as.data.frame(hill_mean_per_hour_per_day)
colnames(hill_mean_per_hour_per_day)  = list("Monday", "Tuesday", "Wednesday","Thursday", "Friday", "Saturday", "Sunday")
hill_mean_per_hour_per_day$time_range = c(1:24)
hill_mean_per_hour_per_day            = melt(hill_mean_per_hour_per_day, id='time_range', variable.name='Days')

hill_mean_per_hour            = as.data.frame(hill_mean_per_hour)
colnames(hill_mean_per_hour)  = list("Weekly")
hill_mean_per_hour$time_range = c(1:24)

#---------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

ggplot() + 
    geom_line(data=hill_mean_per_hour_per_day, aes(x=time_range, y=value, color=Days)) + 
    geom_line(data=hill_mean_per_hour, aes(x=time_range, y=Weekly), linewidth = 1.5)

In [ ]:
# Plotting the loading mean per hour for each day of the week for Valley stations

valley_mean_per_hour_per_day = colMeans(loading_valley)
valley_mean_per_hour_per_day = matrix(valley_mean_per_hour_per_day, nrow = 24)
valley_mean_per_hour         = rowMeans(valley_mean_per_hour_per_day)

#---------------------#

valley_mean_per_hour_per_day            = as.data.frame(valley_mean_per_hour_per_day)
colnames(valley_mean_per_hour_per_day)  = list("Monday", "Tuesday", "Wednesday","Thursday", "Friday", "Saturday", "Sunday")
valley_mean_per_hour_per_day$time_range = c(1:24)
valley_mean_per_hour_per_day            = melt(valley_mean_per_hour_per_day, id='time_range', variable.name='Days')

valley_mean_per_hour            = as.data.frame(valley_mean_per_hour)
colnames(valley_mean_per_hour)  = list("Weekly")
valley_mean_per_hour$time_range = c(1:24)

#---------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

ggplot() + 
    geom_line(data=valley_mean_per_hour_per_day, aes(x=time_range, y=value, color=Days)) + 
    geom_line(data=valley_mean_per_hour, aes(x=time_range, y=Weekly), linewidth = 1.5)

## Influence of localisation

In [ ]:
# Calculating specific hours

hours15 <- seq(15, 168, 24)
hours4 <- seq(4, 168, 24)
average_loading15 <- apply(loading[, hours15], 1, mean)
average_loading4 <- apply(loading[, hours4], 1, mean)

#-----------------------#

# Station dispersion map with color based on average load at 3pm

fig <- plot_ly(coord, type = "scattermapbox", lat = ~latitude, lon = ~longitude, 
                mode = "markers", color = average_loading15,
                text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                hoverinfo = "text") %>%
  layout(mapbox = list(style = "carto-positron"),
         autosize=TRUE,
         title = "Stations loading - average loading at 3pm")
fig

In [ ]:
# Loading data of the Seine

Seine <- read.table("velib_data/Seine.csv", sep = " ",header=TRUE)
head(Seine)

In [ ]:
# Seine Map

fig_seine <- plot_ly(Seine, type = "scattermapbox", lat = ~latitude, lon = ~longitude, 
                     mode = "markers", text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                     hoverinfo = "text") %>%
  layout(mapbox = list(style = "carto-positron"), 
         title = "The Seine")
fig_seine


In [ ]:
# Convert the Seine data to a numeric matrix

Seine_data <- as.matrix(Seine)

#----------------------#

# Calculate the minimum distances from each point to the Seine

max_distances <- sapply(1:nrow(coord), function(i) {
  min(sqrt(rowSums((Seine_data - c(coord$longitude[i], coord$latitude[i]))^2)))
})

In [ ]:
# Representation of stations closest to the Seine with color based on distance

fig_distances <- plot_ly(coord, type = "scattermapbox", lat = ~latitude, lon = ~longitude, 
                         mode = "markers", color = max_distances,
                         text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                         hoverinfo = "text") %>%
  layout(mapbox = list(style = "carto-positron"), 
         title = "Stations closest to the Seine")
fig_distances


In [ ]:
# Convert data structures

hours15 <- seq(15, 167, by = 24)
hours4 <- seq(4, 167, by = 24)
average_loading15 <- apply(loading[, hours15], 1, mean)
average_loading4 <- apply(loading[, hours4], 1, mean)
average_loading <- rowMeans(loading)
max_distances <- sapply(1:nrow(coord), function(i) {
  min(sqrt((coord$longitude[i] - Seine$longitude)^2 + (coord$latitude[i] - Seine$latitude)^2))
})

#---------------------#

# Scatter plots of maximum distances from average load at different times

fig_scatter <- plot_ly(coord, x = ~max_distances, type = "scatter", 
                       mode = "markers", marker = list(size = 10, opacity = 0.5),
                       y = ~average_loading15, color = ~average_loading15,
                       text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                       hoverinfo = "text") %>%
  layout(title = "max_distances vs average loading at 3pm",
         xaxis = list(title = "max_distances"), 
         yaxis = list(title = "average loading"),
         showlegend = TRUE)
fig_scatter


In [ ]:
# Scatter plots of maximum distances from average load at different times

fig_scatter <- plot_ly(coord, x = ~max_distances, type = "scatter", 
                       mode = "markers", marker = list(size = 10, opacity = 0.5),
                       y = ~average_loading4, color = ~average_loading4,
                       text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                       hoverinfo = "text") %>%
  layout(title = "max_distances vs average loading at 4pm",
         xaxis = list(title = "max_distances"), 
         yaxis = list(title = "average loading"),
         showlegend = TRUE)
fig_scatter

In [ ]:
# Scatter plots of maximum distances from average load at different times

fig_scatter <- plot_ly(coord, x = ~max_distances, type = "scatter", 
                       mode = "markers", marker = list(size = 10, opacity = 0.5),
                       y = ~average_loading, color = ~average_loading,
                       text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                       hoverinfo = "text") %>%
  layout(title = "max_distances vs average loading",
         xaxis = list(title = "max_distances"), 
         yaxis = list(title = "average loading"),
         showlegend = TRUE)
fig_scatter

In [ ]:
# Linear regression and p-value extraction

model1 <- lm(max_distances ~ average_loading15)
model2 <- lm(max_distances ~ average_loading)
model3 <- lm(max_distances ~ average_loading4)

print(paste("Valeur p du test de nullité de la pente (average_loading15,max_distances):", summary(model1)$coefficients[2,4]))
print(paste("Valeur p du test de nullité de la pente (average_loading,max_distances):", summary(model2)$coefficients[2,4]))
print(paste("Valeur p du test de nullité de la pente (average_loading4,max_distances):", summary(model3)$coefficients[2,4]))

#---------------------#

# Calculate distances from a central point
center_point <- c(48.8569262793918, 2.3413513652516027)
coord$radius <- sqrt((coord$latitude - center_point[1])^2 + (coord$longitude - center_point[2])^2)

# Plotting
fig <- plot_ly(data = coord, lat = ~latitude, lon = ~longitude, type = 'scattermapbox', mode = 'markers',
               marker = list(size = 10, color = ~radius),
               text = ~paste("Radius:", radius)) %>%
  layout(mapbox = list(style = "carto-positron", zoom = 10, center = list(lat = center_point[1], lon = center_point[2])),
         title = "Stations closest to the city center") %>%
  add_trace(
    type = 'scattermapbox', mode = 'markers',
    lat = c(center_point[1]), lon = c(center_point[2]),
    marker = list(size = 10)
  ) 

fig

In [ ]:
# Additional linear regression with calculated radius

model4 <- lm(coord$radius ~ average_loading)
print(paste("Valeur p du test de nullité de la pente (average_loading,radius):", summary(model4)$coefficients[2,4]))

# <FONT COLOR="Red">ACP and Clustering on transposed data   </font>

## ACP on transposed data

In [ ]:
loading_data_T = as.matrix(t(loading))
rownames(loading_data_T) <- make.unique(rownames(loading_data_T))

pca_T = PCA(loading_data_T, scale.unit = FALSE, graph=FALSE)

#-----------------#

options(repr.plot.width = 15, repr.plot.height = 10)

fviz_eig(pca_T)
fviz_pca_var(pca_T,axes=c(1,2))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

fviz_pca_ind(pca_T,axes=c(1,2),geom=c("point"))

## Clustering on transposed data with k-means

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

# Clustering with kmeans
reskmeans_T <- kmeans(loading_data_T, centers = 2)

# Visualization of clusters
fviz_cluster(reskmeans_T, data = loading_data_T, ellipse.type = "norm", geom = c("point"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

# Clusters on PCA_T
options(repr.plot.width = 12, repr.plot.height = 10)

fviz_pca_ind(pca_T, axes=c(1,2), geom=c("point"), habillage=as.factor(reskmeans_T$cluster))

In [ ]:
# Elbow method used with total within sum of square as metric

fviz_nbclust(loading_data_T, kmeans, method = c("wss"))

In [ ]:
# Elbow method used with silhouette score as metric

fviz_nbclust(loading_data_T, kmeans, method = c("silhouette"))

In [ ]:
# Silhouette plots, according to the number of clusters
options(repr.plot.width = 15, repr.plot.height = 6)

# With 2 clusters
reskmeans_T = kmeans(loading_data_T, centers=2) 
sil = silhouette(reskmeans_T$cluster, dist(loading_data_T))
p1 = fviz_silhouette(sil)

# With 3 clusters
reskmeans_T = kmeans(loading_data_T, centers=3) 
sil = silhouette(reskmeans_T$cluster, dist(loading_data_T))
p2 = fviz_silhouette(sil)

# With 4 clusters
reskmeans_T = kmeans(loading_data_T, centers=4) 
sil = silhouette(reskmeans_T$cluster, dist(loading_data_T))
p3 = fviz_silhouette(sil)

# With 5 clusters
reskmeans_T = kmeans(loading_data_T, centers=5) 
sil = silhouette(reskmeans_T$cluster, dist(loading_data_T))
p4 = fviz_silhouette(sil)

options(repr.plot.width = 15, repr.plot.height = 10)
grid.arrange(p1,p2,p3,p4,ncol=2)

## Clustering on PCA of transpose data

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

# Clustering with kmeans
reskmeans_T <- kmeans(pca_T$ind$coord, centers = 2)

# visualization of clusters
fviz_cluster(reskmeans_T, data = pca_T$ind$coord, ellipse.type = "norm", geom = c("point"))

In [ ]:
# Elbow method used with total within sum of square as metric

fviz_nbclust(pca_T$ind$coord, kmeans, method = c("wss"))

In [ ]:
# Elbow method used with silhouette score as metric

fviz_nbclust(pca_T$ind$coord, kmeans, method = c("silhouette"))

In [ ]:
# Silhouette plots, according to the number of clusters
options(repr.plot.width = 15, repr.plot.height = 6)

# With 2 clusters
reskmeans_T = kmeans(pca_T$ind$coord, centers=2) 
sil = silhouette(reskmeans_T$cluster, dist(pca_T$ind$coord))
p1 = fviz_silhouette(sil)

# With 3 clusters
reskmeans_T = kmeans(pca_T$ind$coord, centers=3) 
sil = silhouette(reskmeans_T$cluster, dist(pca_T$ind$coord))
p2 = fviz_silhouette(sil)

# With 4 clusters
reskmeans_T = kmeans(pca_T$ind$coord, centers=4) 
sil = silhouette(reskmeans_T$cluster, dist(pca_T$ind$coord))
p3 = fviz_silhouette(sil)

# With 5 clusters
reskmeans_T = kmeans(pca_T$ind$coord, centers=5) 
sil = silhouette(reskmeans_T$cluster, dist(pca_T$ind$coord))
p4 = fviz_silhouette(sil)

options(repr.plot.width = 15, repr.plot.height = 10)
grid.arrange(p1,p2,p3,p4,ncol=2)

## n_clusters = 2

In [ ]:
data_pca_T <- pca_T$ind$coord[, 1:2]

# KMeans and labels
n_clusters <- 2

kmeans_pca_T <- kmeans(data_pca_T, centers = n_clusters)
tLabelsPCA_T <- kmeans_pca_T$cluster

kmeans_orig_T <- kmeans(loading_data_T, centers = n_clusters)
tLabels_T <- kmeans_orig_T$cluster

In [ ]:
# Clusters Colors

cmapv <- function(x) {
  colors <- c("purple","yellow")
  return(colors[ceiling(x * (length(colors) - 1)) + 1])
}

colorsPCA_T <- sapply(tLabelsPCA_T, function(i) cmapv(i / (n_clusters -1 )))
colors_T <- sapply(tLabels_T, function(i) cmapv(i / (n_clusters-1 )))

label_color_dict2 <- list('Label 1' = 'yellow','Label 0' = 'purple')

cat("Label-ColorPCA Correspondence:\n")
for (label in names(label_color_dict2)) {
  cat(paste("Label", label, ":", label_color_dict2[[label]], "\n"))
}

#----------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)
                   
# Plotting kemans clusters on individuals in the first principal plane
                   
p1 <- ggplot() +
  geom_point(aes(x = data_pca_T[, 1], y = data_pca_T[, 2], color = colorsPCA_T), alpha = 0.5, size = 2) +
  ggtitle('Projections in 1st factorial plane-Colored w.r.t. kmeans on PCA data') +
  xlab('Composante principale 1') +
  ylab('Composante principale 2') +
  scale_color_identity()

p2 <- ggplot() +
  geom_point(aes(x = data_pca_T[, 1], y = data_pca_T[, 2], color = colors_T), alpha = 0.5, size = 2) +
  ggtitle('Projections in 1st factorial plane-Colored w.r.t. kmeans on original data') +
  xlab('Composante principale 1') +
  ylab('Composante principale 2') +
  scale_color_identity()

grid.arrange(p1, p2, ncol = 2)

In [ ]:
# Time series plot = clusters in function of the time of the day

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

days <- seq(0, 168, length.out = 8)

p3 <- ggplot() +
  geom_line(aes(x = 1:length(tLabels_T), y = tLabels_T)) +
  ggtitle('Clusters on the original Data') +
  geom_vline(xintercept = days, color = 'red', alpha = 0.3)

p4 <- ggplot() +
  geom_line(aes(x = 1:length(tLabelsPCA_T), y = tLabelsPCA_T)) +
  ggtitle('Clusters on the PCA Data') +
  geom_vline(xintercept = days, color = 'red', alpha = 0.3)

grid.arrange(p4, p3, ncol = 2)

## n_clusters = 4

In [ ]:
data_pca_T <- pca_T$ind$coord[, 1:2]

# KMeans and labels
n_clusters <- 4

kmeans_pca_T <- kmeans(data_pca_T, centers = n_clusters)
tLabelsPCA_T <- kmeans_pca_T$cluster

kmeans_orig_T <- kmeans(loading_data_T, centers = n_clusters)
tLabels_T <- kmeans_orig_T$cluster

In [ ]:
# Clusters Colors

cmapv <- function(x) {
  colors <- c("purple", "blue", "green", "yellow")
  return(colors[ceiling(x * (length(colors) - 1)) + 1])
}

colorsPCA_T <- sapply(tLabelsPCA_T, function(i) cmapv(i / (n_clusters -1 )))
colors_T <- sapply(tLabels_T, function(i) cmapv(i / (n_clusters-1 )))

label_color_dict <- list('Label 3' = 'yellow', 'Label 2' = 'green', 'Label 1' = 'blue', 'Label 0' = 'purple')

cat("Label-ColorPCA Correspondence:\n")
for (label in names(label_color_dict)) {
  cat(paste("Label", label, ":", label_color_dict[[label]], "\n"))
}

#----------------------------#
                   
options(repr.plot.width = 15, repr.plot.height = 10)
                   
# Plotting kemans clusters on individuals in the first principal plane
                   
p1 <- ggplot() +
  geom_point(aes(x = data_pca_T[, 1], y = data_pca_T[, 2], color = colorsPCA_T), alpha = 0.5, size = 2) +
  ggtitle('Projections in 1st factorial plane-Colored w.r.t. kmeans on PCA data') +
  xlab('Composante principale 1') +
  ylab('Composante principale 2') +
  scale_color_identity()

p2 <- ggplot() +
  geom_point(aes(x = data_pca_T[, 1], y = data_pca_T[, 2], color = colors_T), alpha = 0.5, size = 2) +
  ggtitle('Projections in 1st factorial plane-Colored w.r.t. kmeans on original data') +
  xlab('Composante principale 1') +
  ylab('Composante principale 2') +
  scale_color_identity()

grid.arrange(p1, p2, ncol = 2)

In [ ]:
# Time series plot = clusters in function of the time of the day

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

days <- seq(0, 168, length.out = 8)

p3 <- ggplot() +
  geom_line(aes(x = 1:length(tLabels_T), y = tLabels_T)) +
  ggtitle('Clusters on the original Data') +
  geom_vline(xintercept = days, color = 'red', alpha = 0.3)

p4 <- ggplot() +
  geom_line(aes(x = 1:length(tLabelsPCA_T), y = tLabelsPCA_T)) +
  ggtitle('Clusters on the PCA Data') +
  geom_vline(xintercept = days, color = 'red', alpha = 0.3)

grid.arrange(p4, p3, ncol = 2)

## Hour profile on original transpose data

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

hour_bins <- seq(-0.5, 24.5, by = 1)
par(mfrow = c(2, 2))

for (label in unique(tLabels_T)) {
  indexes <- which(tLabels_T == label)
  week_hours <- indexes[indexes <= 24 * 5] %% 24
  week_end_hours <- indexes[indexes > 24 * 5] %% 24
  
  hist(week_hours, breaks = hour_bins, col = rgb(0, 0, 1, 0.5), border = 'grey', main = paste('Histogram of hours represented in label', label), xlab = 'Hour', ylab = 'Frequency')
  hist(week_end_hours, breaks = hour_bins, col = rgb(1, 0, 0, 0.5), border = 'grey', add = TRUE)
  legend('topright', legend = c('Week', 'Week-end'), fill = c(rgb(0, 0, 1, 0.5), rgb(1, 0, 0, 0.5)))
}

## Hour profile on PCA of transpose data

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

hour_bins <- seq(-0.5, 24.5, by = 1)
par(mfrow = c(2, 2))

for (label in unique(tLabelsPCA_T)) {
  indexes <- which(tLabels_T == label)
  week_hours <- indexes[indexes <= 24 * 5] %% 24
  week_end_hours <- indexes[indexes > 24 * 5] %% 24
  
  hist(week_hours, breaks = hour_bins, col = rgb(0, 0, 1, 0.5), border = 'grey', main = paste('Histogram of hours represented in label', label), xlab = 'Hour', ylab = 'Frequency')
  hist(week_end_hours, breaks = hour_bins, col = rgb(1, 0, 0, 0.5), border = 'grey', add = TRUE)
  legend('topright', legend = c('Week', 'Week-end'), fill = c(rgb(0, 0, 1, 0.5), rgb(1, 0, 0, 0.5)))
}

# <FONT COLOR="Red">ACP and Clustering on original data   </font>

## ACP on original data

In [ ]:
loading_data = as.matrix(loading)

pca = PCA(loading_data, scale.unit = FALSE, graph=FALSE)

#-------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

fviz_eig(pca)
fviz_pca_var(pca,axes=c(1,2))

## Interpretation of the first principal components

In [ ]:
# PCA individuals in 1st principal plane colored by Hill/Valley

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

fviz_pca_ind(pca,axes=c(1,2),geom=c("point"),habillage=coord$hill)

In [ ]:
data_pca2 = pca$ind$coord[,1:2]

# Normalize the max_distances
norm <- rescale(max_distances, to = c(min(max_distances), max(max_distances)))

# Create colors based on the normalized max_distances
colors <- colorRampPalette(c("blue", "red"))(length(max_distances))

#----------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.5, size = 2, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. max_distances",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(max_distances, data_pca2[, 1]), aes(x = max_distances, y = data_pca2[, 1])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "max_distances vs 1st component",
       x = "max_distances",
       y = "1st component") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#----------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(max_distances, data_pca2[, 1])
print(paste("Correlation coefficient:", correlation_coefficient))

In [ ]:
# Normalize the average_loading
norm <- rescale(average_loading, to = c(min(average_loading), max(average_loading)))

# Create colors based on the normalized max_distances
colors <- colorRampPalette(c("blue", "red"))(length(average_loading))

#----------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.5, size = 2, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. average_loading",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(average_loading, data_pca2[, 1]), aes(x = average_loading, y = data_pca2[, 1])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "average_loading vs 1st component",
       x = "average_loading",
       y = "1st component") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#----------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(average_loading, data_pca2[, 1])
print(paste("Correlation coefficient:", correlation_coefficient))

In [ ]:
# Hour variable in 1st principal plane colored by cluster on T_data

options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

coord1 <- pca$svd$V[,1] * sqrt(pca$eig[1, 1])
coord2 <- pca$svd$V[,2] * sqrt(pca$eig[2, 1])
mask <- ifelse(tLabelsPCA_T %in% c(2, 3), 'red', 'blue')

p <- ggplot() +
  geom_segment(aes(x = 0, y = 0, xend = coord1, yend = coord2, color = mask), alpha = 0.5) +
  ggtitle('Hour variable Graph-colored w.r.t clusters on transposed data')

print(p)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

p1 <- ggplot(data.frame(PC1 = pca$svd$V[,1]), aes(x = 1:length(PC1), y = PC1)) +
  geom_line() +
  geom_vline(xintercept = days, color = 'red', alpha = 0.3) +
  ggtitle('1st principal axis values')

p2 <- ggplot(data.frame(PC2 = pca$svd$V[,2]), aes(x = 1:length(PC2), y = PC2)) +
  geom_line() +
  geom_vline(xintercept = days, color = 'red', alpha = 0.3) +
  ggtitle('2nd principal axis values')

grid.arrange(p1, p2, ncol = 2)

In [ ]:
#Making link between clusters on hour and principal components

high_hours2 <- c()
high_hours <- c()
low_hours <- c()

for (i in 1:length(pca$svd$V[,1])) {
  if (pca$svd$V[i,1] > 0.09) {
    high_hours <- c(high_hours, i %% 24)
  }
}

for (i in 1:length(pca$svd$V[,2])) {
  if (pca$svd$V[i,2] > 0.1) {
    high_hours2 <- c(high_hours2, i %% 24)
  } else if (pca$svd$V[i,2] < -0.05) {
    low_hours <- c(low_hours, i %% 24)
  }
}

#-------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

p1 <- hist(high_hours, breaks = hour_bins, col = rgb(0, 0, 1, 0.5), border = 'grey', main = paste('Important hours for component 1'), xlab = 'Hour', ylab = 'Frequency')
p2 <- hist(high_hours2, breaks = hour_bins, col = rgb(0, 1, 0, 0.5), border = 'grey', main = paste('Hours corresponding to positive coeffs in comp2'), xlab = 'Hour', ylab = 'Frequency')
p3 <- hist(low_hours, breaks = hour_bins, col = rgb(1, 0, 0, 0.5), border = 'grey', main = paste('Hours corresponding to negative coeffs in comp2'), xlab = 'Hour', ylab = 'Frequency')

grid.arrange(p1, p2, p3, ncol = 3)


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

for (i in seq(1, length(unique(tLabels_T)), by = 2)) {
    labels <- unique(tLabels_T)[i:(i + 1)]
    indexes <- which(tLabels_T %in% labels)
    week_hours <- indexes[indexes <= 24 * 5] %% 24
    week_end_hours <- indexes[indexes > 24 * 5] %% 24
    
    hist(week_hours, breaks = hour_bins, col = rgb(0, 0, 1, 0.5), border = 'grey', main = paste('Histogram of hours represented in label', labels), xlab = 'Hour', ylab = 'Frequency')
    hist(week_end_hours, breaks = hour_bins, col = rgb(1, 0, 0, 0.5), border = 'grey', add = TRUE)
    legend('topright', legend = c('Week', 'Week-end'), fill = c(rgb(0, 0, 1, 0.5), rgb(1,0,0,0.5)))
    
}

In [ ]:
# Define the night and day vectors
night <- c(0, 1, 2, 3, 4, 5, 6, 7, 8, 21, 22, 23)
day <- c(10, 11, 12, 13, 14, 15, 16, 17)

# Adjust the index to start from 0 instead of 1
index <- 0:167

# Create the filters
filter_c1 <- ifelse(index %% 24 %in% night, 2, 1)
filter_c2 <- ifelse(index %% 24 %in% day, 1, ifelse(index %% 24 %in% night, -1, 0))
filter_labels <- ifelse(tLabels_T %in% c(0, 1), 1, -1)

# Ensure filter matrices match the dimensions of loading_data
filter_c1_matrix <- matrix(filter_c1, nrow = nrow(loading_data), ncol = length(filter_c1), byrow = TRUE)
filter_c2_matrix <- matrix(filter_c2, nrow = nrow(loading_data), ncol = length(filter_c2), byrow = TRUE)
filter_labels_matrix <- matrix(filter_labels, nrow = nrow(loading_data), ncol = length(filter_labels), byrow = TRUE)

# Calculate the averages using the filters
average_loading <- rowMeans(loading_data)
c1_average <- rowMeans(loading_data * filter_c1_matrix)
c2_average <- rowMeans(loading_data * filter_c2_matrix)
labels_average <- rowMeans(loading_data * filter_labels_matrix)

## 1st principal component approximation

In [ ]:
# Normalize c1_average
norm <- rescale(c1_average, to = c(min(c1_average), max(c1_average)))

# Create colors based on the normalized c1_average
colors <- colorRampPalette(c("blue", "red"))(length(c1_average))

#---------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.9, size = 3, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. c1_average",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(c1_average, data_pca2[, 1]), aes(x = c1_average, y = data_pca2[, 1])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "c1_average vs 1st component",
       x = "c1_average",
       y = "1st component") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#-------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(c1_average, data_pca2[, 1])
print(paste("Correlation coefficient:", correlation_coefficient))

## 2nd principal component approximation

In [ ]:
# Normalize c2_average
norm <- rescale(c2_average, to = c(min(c2_average), max(c2_average)))

# Create colors based on the normalized c2_average
colors <- colorRampPalette(c("blue", "pink"))(length(c2_average))

#------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.9, size = 3, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. c2_average",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(c2_average, data_pca2[, 2]), aes(x = c2_average, y = data_pca2[, 2])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "c2_average vs 2nd component",
       x = "c2_average",
       y = "2nd component") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(c2_average, data_pca2[, 2])
print(paste("Correlation coefficient:", correlation_coefficient))

In [ ]:
# Normalize labels_average
norm <- rescale(labels_average, to = c(min(labels_average), max(labels_average)))

# Create colors based on the normalized labels_average
colors <- colorRampPalette(c("blue", "pink"))(length(labels_average))

#-------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.9, size = 3, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. labels_average",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(labels_average, data_pca2[, 2]), aes(x = labels_average, y = data_pca2[, 2])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "labels_average vs 2nd component",
       x = "labels_average",
       y = "2nd component") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(labels_average, data_pca2[, 2])
print(paste("Correlation coefficient:", correlation_coefficient))

# Clustering on original data with k-means

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

# Reset loading_data dataframe line names
rownames(loading_data) <- NULL

# Clustering with kmeans
reskmeans <- kmeans(loading_data, centers = 2)

# Visualization of clusters
fviz_cluster(reskmeans, data = loading_data, ellipse.type = "norm", geom = c("point"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

# Clusters vs Hill/Valley

grid.arrange(
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(reskmeans$cluster)),
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=coord$hill),
    ncol=2
)

In [ ]:
# Elbow method used with total within sum of square as metric

fviz_nbclust(loading_data, kmeans, method = c("wss"))

In [ ]:
# Elbow method used with silhouette score as metric

fviz_nbclust(loading_data, kmeans, method = c("silhouette"))

In [ ]:
# Silhouette plots, according to the number of clusters
options(repr.plot.width = 15, repr.plot.height = 10)

# With 2 clusters
reskmeans = kmeans(loading_data, centers=2) 
sil = silhouette(reskmeans$cluster, dist(loading_data))
p1 = fviz_silhouette(sil)

# With 3 clusters
reskmeans = kmeans(loading_data, centers=3) 
sil = silhouette(reskmeans$cluster, dist(loading_data))
p2 = fviz_silhouette(sil)

# With 4 clusters
reskmeans = kmeans(loading_data, centers=4) 
sil = silhouette(reskmeans$cluster, dist(loading_data))
p3 = fviz_silhouette(sil)

# With 5 clusters
reskmeans = kmeans(loading_data, centers=5) 
sil = silhouette(reskmeans$cluster, dist(loading_data))
p4 = fviz_silhouette(sil)

grid.arrange(p1,p2,p3,p4,ncol=2)

## Clustering with CAH

In [ ]:
d = dist(loading_data, method="euclidean")

# Clustering

#hclustsingle = hclust(d, method="single")
hclustcomplete = hclust(d, method="complete")
hclustaverage = hclust(d, method="average")

#-------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

#Dendograms visualization

#fviz_dend(hclustsingle, show_labels=FALSE, main='Dendrogram - Single linkage')
fviz_dend(hclustcomplete, show_labels=FALSE, main='Dendrogram - Complete linkage')
fviz_dend(hclustaverage, show_labels=FALSE, main='Dendrogram - Average linkage')

In [ ]:
# Appropriate number of cluster for complete linkage

options(repr.plot.width = 12, repr.plot.height = 6)

#-------------------#

grid.arrange(
    fviz_nbclust(loading_data, FUNcluster=hcut, method="wss") + ggtitle("WSS according to nb of clusters"),
    fviz_nbclust(loading_data, FUNcluster=hcut, method="silhouette") + ggtitle("silhouette according to nb of clusters"),
    ncol=2
)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

#-------------------#

reshclust3 = cutree(hclustcomplete, 3)
reshclust4 = cutree(hclustcomplete, 4)

fviz_dend(hclustcomplete, k=3, show_labels=FALSE, rect=TRUE)
fviz_dend(hclustcomplete, k=4, show_labels=FALSE, rect=TRUE)

## Clustering with GMM

In [ ]:
# impossible d'importer la librarie mclust
# faire clustering avec GMM -> BIC et ICL
# faire la comparaison avec Kmeans et CAH

## 3 clusters 

In [ ]:
# KMeans with 3 clusters

n_clusters <- 3
kmeans_result <- kmeans(loading_data, centers = n_clusters)
Labels3 <- kmeans_result$cluster
coord$Labels3 <- Labels3

In [ ]:
# Visualization of the 3 clusters in PCA plan
Labels3 <- kmeans_result$cluster
fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(Labels3))

In [ ]:
# Kmeans vs CAH clusters

tbl = table(Labels3, reshclust3)
print(tbl)

options(repr.plot.width = 10, repr.plot.height = 10)

mosaicplot(tbl, color=c(2:4))

#---------------------#

options(repr.plot.width = 20, repr.plot.height = 10)

grid.arrange(
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(reshclust3)),
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(Labels3)),
    ncol=2
)

### Separation between class 0 and 1 with along first principal axis

In [ ]:
# Loading data analysis
loading_1 <- loading_data[Labels3 == 1, ]
loading_2 <- loading_data[Labels3 == 2, ]

meanR_1 <- rowMeans(loading_1 * filter_c1)
meanR_2 <- rowMeans(loading_2 * filter_c1)
meanR_1_sorted <- sort(meanR_1)
meanR_2_sorted <- sort(meanR_2)

par(mfrow = c(1, 2))
plot(meanR_2_sorted, type = 'p', col = 'blue', pch = 16, cex = 0.5, xlab = 'Stations', ylab = 'Average loading', main = "c1_average value per station - Class0 vs. Class1")
plot(meanR_1_sorted, type = 'p', col = 'red', pch = 16, cex = 0.5, xlab = 'Stations', ylab = 'Average loading', main = "c1_average value per station - Class0 vs. Class1")
#points(seq_along(meanR_1_sorted),meanR_1_sorted, col = 'red', pch = 16, cex = 0.5)

hist(meanR_2, breaks = 20, freq = FALSE, col = 'blue', main = 'Histograms of c1_average - Class0 vs. Class1', xlab = 'Average Loading', ylab = 'Density')
hist(meanR_1, breaks = 20, freq = FALSE, col = rgb(1, 0, 0, 0.5), add = TRUE)

### Separation between class 0 and 2 / between 1 and 2 with along diagonal axis

In [ ]:
# PCA components analysis

axis0_2 <- (pca$svd$V[, 2] - pca$svd$V[, 1]) / sqrt(sum((pca$svd$V[, 2] - pca$svd$V[, 1])^2))
axis1_2 <- (pca$svd$V[, 1] + pca$svd$V[, 2]) / sqrt(sum((pca$svd$V[, 1] + pca$svd$V[, 2])^2))

par(mfrow = c(1, 2))
plot(axis0_2, type = 'l', main = 'Class 0 vs 2 separation axis')
abline(v = days, col = 'red', lty = 2, lwd = 0.5)

plot(axis1_2, type = 'l', main = 'Class 1 vs 2 separation axis')
abline(v = days, col = 'red', lty = 2, lwd = 0.5)

In [ ]:
# Filter analysis

night <- c(0, 1, 2, 3, 4, 5, 6, 7, 8)
day <- setdiff(0:23, night)
evening <- c(18, 19, 20, 21, 22, 23)

filter_0_2 <- sapply(0:167, function(i) if (i %% 24 %in% night) -2 else if (i %% 24 %in% day && i %/% 24 < 5) 1 else 0)
filter_1_2 <- sapply(0:167, function(i) if (i %% 24 %in% evening || i > 115) 1 else if (i %% 24 %in% day) 2 else 0)

scaled_axis_0_2 <- (axis0_2 - min(axis0_2)) / (max(axis0_2) - min(axis0_2)) * (1 - -2) + -2
scaled_axis_1_2 <- (axis1_2 - min(axis1_2)) / (max(axis1_2) - min(axis1_2)) * 2

par(mfrow = c(1, 2))
plot(scaled_axis_0_2, type = 'l', main = 'filter_0_2')
lines(filter_0_2, col = 'red')

plot(scaled_axis_1_2, type = 'l', main = 'filter_1_2')
lines(filter_1_2, col = 'red')

average_0_2 <- rowMeans(loading_data * filter_0_2)
average_1_2 <- rowMeans(loading_data * filter_1_2)

In [ ]:
# Normalize average_0_2
norm <- rescale(average_0_2, to = c(min(average_0_2), max(average_0_2)))

# Create colors based on the normalized average_0_2
colors <- colorRampPalette(c("blue", "pink"))(length(average_0_2))

#---------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.5, size = 2, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. average_0_2",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(average_0_2, data_pca2[,2] - data_pca2[, 1]), aes(x = average_0_2, y = data_pca2[,2] - data_pca2[, 1])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "average loading vs 0/2 Separation axis",
       x = "average_0_2",
       y = "0_2 Separation variable") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#-------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(average_0_2, data_pca2[,2] - data_pca2[, 1])
print(paste("Correlation coefficient:", correlation_coefficient))

In [ ]:
# Normalize average_0_2
norm <- rescale(average_1_2, to = c(min(average_1_2), max(average_1_2)))

# Create colors based on the normalized average_1_2
colors <- colorRampPalette(c("blue", "pink"))(length(average_1_2))

#---------------------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plot the first subplot
p1 <- ggplot(data_pca2, aes(x = data_pca2[, 1], y = data_pca2[, 2])) +
  geom_point(alpha = 0.5, size = 2, color = colors) +
  labs(title = "Projection on 1st plane - Colored w.r.t. average_1_2",
       x = "Principal Component 1",
       y = "Principal Component 2") +
  theme_bw() +
  theme(panel.grid.major = element_blank(),
        panel.grid.minor = element_blank())

# Plot the second subplot
p2 <- ggplot(data.frame(average_1_2, data_pca2[,2] + data_pca2[, 1]), aes(x = average_1_2, y = data_pca2[,2] + data_pca2[, 1])) +
  geom_point(alpha = 0.5, size = 2) +
  labs(title = "average loading vs 1/2 Separation axis",
       x = "average_1_2",
       y = "1_2 Separation variable") +
  theme_bw()

grid.arrange(p1, p2, ncol = 2)

#-------------------------#

# Calculate the correlation coefficient
correlation_coefficient <- cor(average_1_2, data_pca2[,2] + data_pca2[, 1])
print(paste("Correlation coefficient:", correlation_coefficient))

## 4 clusters 

In [ ]:
# KMeans with 4 clusters

n_clusters <- 4
kmeans_result <- kmeans(loading_data, centers = n_clusters)
Labels4 <- kmeans_result$cluster
coord$Labels4 <- Labels4

In [ ]:
# Visualization of the 4 clusters in PCA plan

Labels4 <- kmeans_result$cluster
fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(Labels4))

In [ ]:
# Kmeans vs CAH clusters

tbl = table(Labels4, reshclust4)
print(tbl)

options(repr.plot.width = 10, repr.plot.height = 10)

mosaicplot(tbl, color=c(2:4))

#---------------------#

options(repr.plot.width = 20, repr.plot.height = 10)

grid.arrange(
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(reshclust4)),
    fviz_pca_ind(pca, axes=c(1,2), geom=c("point"), habillage=as.factor(Labels4)),
    ncol=2
)

# Interpretation of Station profiles and user patterns

In [ ]:
# Station dispersion map with color based on average load at 3pm

fig <- plot_ly(coord, type = "scattermapbox", lat = ~latitude, lon = ~longitude, 
                mode = "markers", color = Labels4,
                text = ~paste("Longitude: ", longitude, "<br>Latitude: ", latitude),
                hoverinfo = "text") %>%
  layout(mapbox = list(style = "carto-positron"),
         autosize=TRUE,
         title = "Stations loading - average loading at 3pm")
fig

In [ ]:
plots <- list()

for (k in 1:4) {
  # Subset data for the current cluster
  data_subset <- loading_data[Labels4 == k, ]
  
  # Calculate the average value of the row means for the subset
  avg_value <- mean(rowMeans(data_subset))
  
  # Create a plot for the current cluster
  p <- ggplot(data = data.frame(x = 1:ncol(data_subset), y = colMeans(data_subset)), aes(x = x, y = y)) +
    geom_line() +
    geom_hline(yintercept = avg_value, color = 'red', linetype = 'dashed') +
    scale_x_continuous(breaks = seq(0, 168, by = 24)) +
    labs(y = 'Loading', title = paste('Average loading in cluster', k))
  
  # Store the plot in the list
  plots[[k]] <- p
}

# Arrange the plots in a 2x2 grid
grid.arrange(grobs = plots, ncol = 2, nrow = 2)


# <FONT COLOR="Red">Clustering with MCA data   </font>

In [ ]:
# Histogram of max_distances
hist(max_distances,bins=20)

In [ ]:
# Data loading
load <- read.csv("velib_data/velibLoading.csv", sep = " ")
coord <- read.csv("velib_data/velibCoord.csv", sep = " ")

#--------------#

num_subdivisions <- 5

categorical_data <- data.frame(matrix(ncol = 0, nrow = nrow(load)))

for (col in colnames(load)) {
  subdivisions <- cut(load[[col]], breaks = num_subdivisions, labels = FALSE)
  categorical_labels <- as.character(subdivisions)
  temp_df <- data.frame(categorical_labels)
  colnames(temp_df) <- col
  categorical_data <- cbind(categorical_data, temp_df)
}

# bonus column
if (is.data.frame(coord) && "bonus" %in% colnames(coord)) {
  categorical_data$bonus <- ifelse(coord$bonus == 1, 1, 0)
} else {
  stop("Le dataframe 'coord' est vide ou n'a pas la colonne 'bonus'")
}

# seine column
if (length(max_distances) == nrow(load)) {
  categorical_data$seine <- ifelse(max_distances < 0.015, 1, 0)
} else {
  stop("La longueur de 'max_distances' ne correspond pas au nombre de lignes de 'load'")
}

#----------------#

head(categorical_data)

In [ ]:
# MCA
mca <- MCA(as.matrix(categorical_data), ncp = 5, graph = FALSE)
row_coordinates <- mca$ind$coord

#-----------------#

options(repr.plot.width = 15, repr.plot.height = 10)

# Plotting cumulative explained variance ratio
explained_variance <- cumsum(mca$eig[, 2])
ggplot(data.frame(Components = 1:length(explained_variance), Variance = explained_variance), aes(x = Components, y = Variance)) +
  geom_line() + geom_point() +
  labs(title = "Cumulative Explained Variance Ratio", x = "Number of Components", y = "Cumulative Explained Variance Ratio") +
  theme_minimal()

In [ ]:
# Plotting individuals in MCA space colored by bonus

options(repr.plot.width = 15, repr.plot.height = 10)

categorical_data$bonus <- as.factor(categorical_data$bonus)

#-----------#

# Plotting individuals in MCA space colored by bonus
fviz_mca_ind(mca, geom = "point", col.ind = categorical_data$bonus) + 
  scale_color_manual(values = c("blue", "red")) +
  labs(title = "Individuals in MCA Space Colored w.r.t. bonus")

In [ ]:
# Plot individuals in MCA space colored by seine

options(repr.plot.width = 15, repr.plot.height = 10)

categorical_data$seine <- as.factor(categorical_data$seine)

#-------------#

fviz_mca_ind(mca, geom = "point", col.ind = categorical_data$seine) + 
  scale_color_manual(values = c("blue", "red")) +
  labs(title = "Individuals in MCA Space Colored w.r.t. seine")

In [ ]:
# Elbow method for KMeans clustering

wcss <- numeric()
for (i in 2:11) {
  kmeans_result <- kmeans(row_coordinates, centers = i, nstart = 25)
  wcss <- c(wcss, kmeans_result$tot.withinss)
}

#-------------#

options(repr.plot.width = 10, repr.plot.height = 10)

ggplot(data.frame(Clusters = 2:11, WCSS = wcss), aes(x = Clusters, y = WCSS)) +
  geom_line() + geom_point() +
  labs(title = "Elbow Method", x = "Number of clusters", y = "WCSS") +
  theme_minimal()

In [ ]:
# Silhouette score for kmeans clustering

options(repr.plot.width = 10, repr.plot.height = 10)

#-------------#

par(mfrow = c(4, 2))
for (k in 2:9) {
  kmeans_result <- kmeans(row_coordinates, centers = k, nstart = 25)
  silhouette_result <- silhouette(kmeans_result$cluster, dist(row_coordinates))
  plot(silhouette_result, main = paste("Silhouette plot for k =", k))
}

In [ ]:
# KMeans clustering with 4 clusters

n_clusters <- 4
kmeans_result <- kmeans(row_coordinates, centers = n_clusters, nstart = 25)
MCALabels <- kmeans_result$cluster

In [ ]:
# Plot kmeans clusters in the MCA plane

options(repr.plot.width = 10, repr.plot.height = 10)

# Plot kmeans clusters in the MCA plane
fviz_mca_ind(mca, geom = "point", col.ind = as.factor(MCALabels)) +
  scale_color_manual(values = rainbow(n_clusters)) +
  labs(title = "Clusters in the MCA Plane")

#-------------#

ggplot(data.frame(row_coordinates), aes(x = Dim.1, y = Dim.2, color = as.factor(MCALabels))) +
  geom_point(alpha = 0.5) +
  labs(title = "Clusters in the MCA Plane", x = "Composante principale 1", y = "Composante principale 2") +
  scale_color_manual(values = rainbow(n_clusters)) +
  theme_minimal()

In [ ]:
# Print percentages

categorical_data$seine <- as.numeric(as.character(categorical_data$seine))

cat(sprintf("%.2f percent of the altitude stations are labeled 2\n", sum(coord$bonus * (MCALabels == 2)) / sum(coord$bonus) * 100))
cat(sprintf("%.2f percent of the stations close to the seine are labeled 0 or 3\n", sum(categorical_data$seine * (MCALabels %in% c(0, 3))) / sum(categorical_data$seine) * 100))

In [ ]:
norm <- function(x, min_val, max_val) {
  (x - min_val) / (max_val - min_val)
}

min_val <- min(MCALabels)
max_val <- max(MCALabels)
norm_labels <- norm(MCALabels, min_val, max_val)

colors <- sapply(unique(MCALabels), function(label) {
  cmapv(norm(label, min_val, max_val))
})

legend_dict <- setNames(colors, unique(MCALabels))

#-----------------#

# Plot MCA and PCA clusters

options(repr.plot.width = 15, repr.plot.height = 10)

par(mfrow = c(1, 2))
for (label in unique(MCALabels)){
  indicesMCA <- MCALabels == label
  indicesPCA <- Labels4 == label
  plot(data_pca2[indicesMCA, 1], data_pca2[indicesMCA, 2], col = legend_dict[[label]], pch = 16, main = "MCA Clusters in the PCA Plane", xlab = "Composante principale 1", ylab = "Composante principale 2")
  plot(data_pca2[indicesPCA, 1], data_pca2[indicesPCA, 2], col = legend_dict[[label]], pch = 16, main = "PCA Clusters in the PCA Plane", xlab = "Composante principale 1", ylab = "Composante principale 2")
}

In [ ]:
# Correspondence between clusters
correspondance <- c(2, 3, 1, 4)

#------------#

options(repr.plot.width = 15, repr.plot.height = 10)

plots <- list()

for (k in 1:4) {
  mean_mca <- colMeans(load[MCALabels == k, ])
  mean_labels4 <- colMeans(load[Labels4 == correspondance[k], ])
  avg_value <- mean(mean_mca)
  
  p <- ggplot() +
    geom_line(aes(x = 1:ncol(load), y = mean_mca), color = 'cyan') +
    geom_line(aes(x = 1:ncol(load), y = mean_labels4), color = 'pink') +
    geom_hline(aes(yintercept = avg_value), color = 'red', linetype = 'dashed') +
    scale_x_continuous(breaks = seq(0, 168, by = 24)) +
    labs(y = 'Loading', title = paste('Average loading in cluster', k)) +
    theme_minimal()
  
  plots[[k]] <- p
}


grid.arrange(grobs = plots, ncol = 2, nrow = 2)
